In [ ]:
import bilby
import glob
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import h5py

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np

from utils import EVENTS
from thesis_utils.plotting import set_plotting, save_figure

set_plotting()

In [ ]:
paths = ["../analysis/IMRPhenomXPHM/", "../analysis/IMRPhenomXP/"]
labels = ["XPHM", "XP"]

In [ ]:
results = {}
for label, path in zip(labels, paths):
    data = {}
    for event in EVENTS:
        data[event] = {"sampling_time": [], "likelihood_evaluations": []}
        result_files = glob.glob(
            f"{path}/outdir_nessai_gwtc_1_{event}_*{label}/result/*par*.hdf5"
        )
        for rf in result_files:
            if os.path.getsize(rf) == 0:
                data[event]["sampling_time"].append(np.nan)
                data[event]["likelihood_evaluations"].append(np.nan)
                continue
            with h5py.File(rf, "r") as f:
                sampling_time = f["sampling_time"][()]
                likelihood_evaluations = f["num_likelihood_evaluations"][()]
            data[event]["sampling_time"].append(sampling_time)
            data[event]["likelihood_evaluations"].append(likelihood_evaluations)
    results[label] = data

In [ ]:
nessai_results = {}
for label, path in zip(labels, paths):
    data = {}
    for event in EVENTS:
        data[event] = {
            "training_time": [],
            "population_time": [],
            "likelihood_evaluation_time": [],
            "sampling_time": [],
            "likelihood_evaluations": [],
        }
        result_files = glob.glob(
            f"{path}/outdir_nessai_gwtc_1_{event}_*{label}/result/*_nessai/result.hdf5"
        )
        for rf in result_files:
            if os.path.getsize(rf) == 0:
                data[event]["sampling_time"].append(np.nan)
                data[event]["training_time"].append(np.nan)
                data[event]["population_time"].append(np.nan)
                data[event]["likelihood_evaluation_time"].append(np.nan)
                data[event]["likelihood_evaluations"].append(np.nan)
                continue
            with h5py.File(rf, "r") as f:
                data[event]["sampling_time"].append(f["sampling_time"][()])
                data[event]["training_time"].append(f["training_time"][()])
                data[event]["population_time"].append(
                    f["population_time"][()] - f["likelihood_evaluation_time"][()]
                )
                data[event]["likelihood_evaluations"].append(
                    f["total_likelihood_evaluations"][()]
                )
                data[event]["likelihood_evaluation_time"].append(
                    f["likelihood_evaluation_time"][()]
                )
    nessai_results[label] = data

In [ ]:
results["XPHM"]["GW150914"]["sampling_time"]

In [ ]:
events = list(results[labels[0]].keys())

In [ ]:
import seaborn as sns

event_colours = sns.color_palette("colorblind", n_colors=len(events))

In [ ]:
analysis_markers = [
    10 * ["s"],
    10 * ["o"],
]

In [ ]:
from matplotlib.lines import Line2D

In [ ]:
legend_elements = []
for event, event_colour, marker in zip(EVENTS, event_colours, analysis_markers[0]):
    legend_elements.append(
        Line2D([], [], color=event_colour, marker="None", label=event, ls="-", lw=2.0)
    )

legend_elements.append(
    Line2D([], [], color="k", marker="o", label="IMRPhenomXP", ls="None"),
)
legend_elements.append(
    Line2D([], [], color="k", marker="s", label="IMRPhenomXPHM", ls="None")
)
legend_elements += 8 * [Line2D([], [], marker="", ls="None")]

In [ ]:
fig = plt.figure()
for (label, data), markers in zip(results.items(), analysis_markers):
    for event, event_colour, marker in zip(events, event_colours, markers):
        times = np.array(data[event]["sampling_time"]) / 3600
        evals = np.array(data[event]["likelihood_evaluations"])

        plt.plot(times, evals, color=event_colour, marker=marker)
plt.legend(handles=legend_elements, ncol=2, loc=2)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Wall time [hrs]")
plt.ylabel("Likelihood evaluations")
plt.show()
save_figure(fig, "timing_XP_vs_XPHM", "figures/timing/")

In [ ]:
hatch = {
    "training": "||",
    "likelihood": "//",
    "population": "\\",
}

In [ ]:
ratios = {}
for analysis, event_data in nessai_results.items():
    ratios[analysis] = {}
    for event, data in event_data.items():
        ratios[analysis][event] = {}
        ratios[analysis][event]["likelihood"] = np.array(
            data["likelihood_evaluation_time"]
        ) / np.array(data["sampling_time"])
        ratios[analysis][event]["population"] = np.array(
            data["population_time"]
        ) / np.array(data["sampling_time"])
        ratios[analysis][event]["training"] = np.array(
            data["training_time"]
        ) / np.array(data["sampling_time"])

In [ ]:
labels = {
    "likelihood": "Likelihood",
    "population": "Population",
    "training": "Training",
}

In [ ]:
def lighten_colour(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    See: https://gist.github.com/ihincks/6a420b599f43fcd7dbd79d56798c4e5a

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys

    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [ ]:
analyses = ["XP", "XPHM"]

In [ ]:
legend_elements = []
fraction = 1.0
for i, k in enumerate(ratios["XPHM"]["GW150914"].keys()):
    legend_elements.append(
        Patch(facecolor=lighten_colour("grey", fraction), label=labels.get(k))
    )
    fraction -= 1 / 3.5

In [ ]:
event_durations = {
    "GW150914": 4,
    "GW151012": 8,
    "GW151226": 16,
    "GW170104": 8,
    "GW170608": 16,
    "GW170729": 4,
    "GW170809": 4,
    "GW170814": 4,
    "GW170818": 4,
    "GW170823": 4,
}

In [ ]:
event_order = sorted(event_durations, key=event_durations.get)

In [ ]:
events_4s = event_order[:6]
events_8s = event_order[6:8]
events_16s = event_order[8:]

In [ ]:
fig, axs = plt.subplots(
    2,
    3,
    sharex="col",
    sharey=True,
    gridspec_kw=dict(
        width_ratios=[6, 2, 2],
        wspace=0.1,
    ),
)
colours = {
    e: c for e, c in zip(events, sns.color_palette("colorblind", n_colors=len(events)))
}
offset = [-0.3, 0.3]
width = 0.6
for m, subset_events in enumerate([events_4s, events_8s, events_16s]):
    xticks = 2 * np.arange(len(subset_events))
    for n, analysis in enumerate(analyses):
        for i, event in enumerate(subset_events):
            for j in range(2):
                bottom = 0
                fraction = 1.0
                for key, data in ratios[analysis][event].items():
                    axs[n, m].bar(
                        xticks[i] + offset[j],
                        data[j],
                        facecolor=lighten_colour(colours[event], fraction),
                        # hatch=hatch.get(key),
                        bottom=bottom,
                        width=width,
                    )
                    bottom += data[j]
                    fraction -= 1 / 3.5

        axs[n, m].set_xticks(xticks, labels=subset_events, rotation=45)
        axs[n, m].tick_params(which="minor", top=False, bottom=False)

axs[0, 0].set_ylabel("Fraction of time")
axs[1, 0].set_ylabel("Fraction of time")

axs[0, 0].set_title(r"\texttt{IMRPhenomXP}", loc="left")
axs[1, 0].set_title(r"\texttt{IMRPhenomXPHM}", loc="left")

axs[0, 0].text(
    0.5, 1.05, r"$4\;\textrm{s}$", transform=axs[0, 0].transAxes, ha="center"
)
axs[0, 1].text(
    0.5, 1.05, r"$8\;\textrm{s}$", transform=axs[0, 1].transAxes, ha="center"
)
axs[0, 2].text(
    0.5, 1.05, r"$16\;\textrm{s}$", transform=axs[0, 2].transAxes, ha="center"
)

fig.legend(handles=legend_elements, ncol=3, loc="center", bbox_to_anchor=(0.5, -0.1))
# plt.yscale("log")
plt.tight_layout()

save_figure(fig, "fraction_of_time", "figures/timing/")
plt.show()